In [1]:
import os
from os.path import join
import csv
import cv2
import numpy as np

from keras.layers import Input, Conv2D, Dropout, MaxPool2D, GlobalAveragePooling2D, GlobalMaxPooling2D, LeakyReLU
from keras.layers import concatenate, Dense
from keras.models import Model
from keras.utils import to_categorical
from keras.callbacks import ReduceLROnPlateau

from paths import PATH_DATA, PATH_PROJECT
from iterators import roundrobin, repeat_infinitely

Using TensorFlow backend.


In [2]:
class DarknetBlock:
    def __init__(self, filters1, filters3, strides=(1, 1)):
        self._filters1 = filters1
        self._filters3 = filters3
        self._strides = strides
        
    def __call__(self, input_layer, *args, **kwargs):
        x = input_layer
        for filters, kernel, strides in zip([self._filters1, self._filters3],
                                            [(1, 1), (3, 3)],
                                            [(1, 1), self._strides]):
            x = Conv2D(filters, kernel, padding='same', strides=strides)(x)
            x = LeakyReLU()(x)
        return x


def build_model(class_count, is_train=False):
    input_tensor = Input(shape=(1, None, None))
    x = Conv2D(10, (3, 3), strides=(2, 2), padding='same')(input_tensor)
    filters = [20, 30, 40]
    for idx, filter_size in enumerate(filters):
        x = DarknetBlock(filter_size, filter_size * 2)(x)
        x = DarknetBlock(filter_size, filter_size * 2)(x)
        if idx < len(filters) - 1:
            x = MaxPool2D(pool_size=(2, 2), padding='same')(x)
            if is_train:
                x = Dropout(0.1)(x)
    
    x = Conv2D(filters[-1], (1, 1), padding='same')(x)
    max_output = GlobalMaxPooling2D()(x)
    average_output = GlobalAveragePooling2D()(x)
    x = concatenate(inputs=[max_output, average_output], axis=1)
    x = Dense(class_count, activation='softmax')(x)
    return Model(inputs=[input_tensor], outputs=[x])

In [3]:
CLASS_COUNT = 6
# cannot make bigger batches because images are of different size
BATCH_SIZE = 1
TRAIN_SIZE = 7000
TEST_SIZE = 1750

In [4]:
def read_label_map(path):
    with open(path, 'r') as fin:
        reader = csv.reader(fin, delimiter=',')
        # ignore header
        next(reader)
        label_map = {filename: int(label) for filename, label in reader}
    return label_map


def get_images_paths(root, label_count):
    dir_paths = list(map(lambda label: join(root, str(label)), range(label_count)))
    file_paths = [list(map(lambda filename: join(dir_path, filename), os.listdir(dir_path))) for dir_path in dir_paths]
    return roundrobin(*file_paths)


def get_train_data(path_images, label_count, label_map):
    paths = repeat_infinitely(get_images_paths, path_images, label_count)
    for path in paths:
        image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            raise ValueError(f'Image cannot be read: {path}')
        yield (np.expand_dims(image, 0),
               to_categorical(label_map[os.path.basename(path)], label_count)[0])


def collect_batches(iterable, batch_size=32, randomize=False, probability=0.5, rotate=False):
    while True:
        images, labels = [], []
        while len(images) < batch_size:
            image, label = next(iterable)
            if not randomize or np.random.rand() < probability:
                if rotate:
                    image = np.rot90(image, np.random.randint(0, 4), (1, 2))
                images.append(image)
                labels.append(label)
        yield np.array(images), np.array(labels)

In [5]:
label_map = read_label_map(join(PATH_DATA, 'train_labels.csv'))
gen_data = get_train_data(join(PATH_DATA, 'train'), CLASS_COUNT, label_map)
gen_batches = collect_batches(gen_data, BATCH_SIZE, randomize=True, rotate=True)

# x, y = next(gen_batches)
# print(y)
# 
# from matplotlib import pyplot as plt
# plt.imshow(x[0, 0], cmap='Greys_r')
# plt.show()

In [6]:
model = build_model(CLASS_COUNT, is_train=True)
model.compile(optimizer='adadelta', loss='categorical_crossentropy')
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 1, None, None) 0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 10, None, None 100         input_1[0][0]                    
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (None, 20, None, None 220         conv2d_1[0][0]                   
____________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)        (None, 20, None, None 0           conv2d_2[0][0]                   
___________________________________________________________________________________________

In [7]:
history = model.fit_generator(generator=gen_batches,
                              steps_per_epoch=TRAIN_SIZE,
                              epochs=50)

Epoch 1/50
7000/7000 [==============================] - 219s - loss: 0.0550   
Epoch 2/50
1008/7000 [===>..........................] - ETA: 140s - loss: 0.0260

KeyboardInterrupt: 

In [ ]:
path_weights = join(PATH_PROJECT, 'weights')
model.save_weights(path_weights, overwrite=True)

In [ ]:
def get_test_data(path_images):
    paths = map(lambda filename: join(path_images, filename), os.listdir(path_images))
    return np.array([list(map(lambda path: cv2.imread(path, cv2.IMREAD_GRAYSCALE), paths))])

In [ ]:
model_predict = build_model(CLASS_COUNT, is_train=False)
model_predict.load_weights(path_weights)

In [ ]:
path_test = join(PATH_DATA, 'test')
answers = {}
for filename in os.listdir(path_test):
    file_path = join(path_test, filename)
    image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    tensor = image.reshape((1, 1, *image.shape))
    label_categorical = model_predict.predict(tensor)
    print(label_categorical)
    answers[filename] = np.argmax(label_categorical)

In [ ]:
with open('submission.csv', 'w') as fout:
    for filename, label in answers.items():
        print(filename, label, sep=',')